In [1]:
import torch     # PyTorch
import datasets  # Huggingface Datasets; actual data

from IPython.display import display  # image display
import matplotlib.pyplot as plt      # image display
%matplotlib inline                   
import random
#Torchvision functions
from torchvision.transforms import ToTensor
import torchvision.transforms.functional as functional
import torchvision.transforms as transforms

In [2]:
dataset = datasets.load_dataset("DrishtiSharma/Anime-Face-Dataset")

Found cached dataset parquet (/Users/boranollana/.cache/huggingface/datasets/DrishtiSharma___parquet/DrishtiSharma--Anime-Face-Dataset-1886e0ceeb6ab6bb/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

dont run the next block, torch.(con)cat fucks up with large tensors. use tim's

In [ ]:
#loading all images from dictionary to list
#all_images = dataset['train'][:63565]['image']
#resizing the first image
#resized_image = functional.resize(all_images[0], size=[64])
#saving the image to a torch tensor
#train_tensor = train_tensor = ToTensor()(resized_image).unsqueeze(0)
#for the rest of the images, resize then concatinate to that tensor, using dimension 0.
for i in range(27528, len(all_images)):
    resized_image = functional.resize(all_images[i], size=[64])
    train_tensor = torch.cat((train_tensor,ToTensor()(resized_image).unsqueeze(0)), 0)
    #if i%10000 == 0:  #I used these as a sanity check for my loop, ignore
    #    print('i = ', i)

#Saving the tensor so we dont run the above code multiple times.
torch.save(train_tensor, 'images.pt')
#If everything went well, we will see a tensor of size [63565, 3, 64, 64]:
train_tensor

In [3]:
# Set the output size
output_size = (64, 64)

# Define a transform to resize the images
transform = transforms.Compose([
    transforms.Resize(output_size),
    transforms.ToTensor(),
])

# Create an empty list to store the resized images as tensors
resized_images = []

# Loop through each image in the dataset and resize it
for i in range( len(dataset['train']) ):
    image = dataset['train'][i]['image']  # Get the i-th image and its label
    image_resized = transform(image)  # Resize the image
    resized_images.append(image_resized)  # Add the resized image to the list
    
    if i % 1000 == 0:
        print("Making progress at ",i,"out of ",len(dataset['train']))

# Convert the list of tensors to a tensor stack
resized_images_tensor = torch.stack(resized_images)

Making progress at  0 out of  63565
Making progress at  1000 out of  63565
Making progress at  2000 out of  63565
Making progress at  3000 out of  63565
Making progress at  4000 out of  63565
Making progress at  5000 out of  63565
Making progress at  6000 out of  63565
Making progress at  7000 out of  63565
Making progress at  8000 out of  63565
Making progress at  9000 out of  63565
Making progress at  10000 out of  63565
Making progress at  11000 out of  63565
Making progress at  12000 out of  63565
Making progress at  13000 out of  63565
Making progress at  14000 out of  63565
Making progress at  15000 out of  63565
Making progress at  16000 out of  63565
Making progress at  17000 out of  63565
Making progress at  18000 out of  63565
Making progress at  19000 out of  63565
Making progress at  20000 out of  63565
Making progress at  21000 out of  63565
Making progress at  22000 out of  63565
Making progress at  23000 out of  63565
Making progress at  24000 out of  63565
Making progre

In [4]:
#saving tensor
torch.save(resized_images_tensor, 'images.pt')
resized_images_tensor.shape

torch.Size([63565, 3, 64, 64])

In [2]:
#loading tensor:
data = torch.load('./images.pt')
data.shape

torch.Size([63565, 3, 64, 64])

In [3]:
#additional imports:
from torch.utils.data import DataLoader

In [4]:
#some hyperparam init: #wandb script here
bs = 5000

For GANs we dont need a train/test split.

In [6]:
#creating dataloader from our tensor
images = DataLoader(data, batch_size=bs, shuffle=False)
#batch shape:
next(iter(images)).shape

torch.Size([5000, 3, 64, 64])

Our discriminator is basically a CNN, whereas our generator is basically the equivalent of a cnn but instead of convolutions, we have the gradients of those convolutions with respect to our inputs;

Discriminator: Conv2d
Generator: ConvTranspose2d


In [35]:
nl = 3
hs = 64
#new dependency:
from torch import nn as nn

class Discriminator(nn.Module):
    def __init__(self, nl, hs):
        super().__init__()
        self.layers = nn.ModuleList()

        self.layers.append(nn.Sequential(
                nn.Conv2d(3, hs, kernel_size=3, stride=1, padding=0),
                nn.BatchNorm2d(hs),
                nn.ReLU()
            ))
        
        for i in range(1, nl-1):
            self.layers.append(nn.Sequential(
                #previous hidden layer size, next hidden layer size from array passed as argument to the constructor
                nn.Conv2d(hs*i, hs*i*2, kernel_size=3, stride=1, padding=0), 
                nn.BatchNorm2d(hs*i*2),
                nn.ReLU()
            ))
        
        self.layers.append(nn.Sequential(
                nn.Conv2d(hs*(nl-1), 1,  kernel_size=3, stride=1, padding=0),
                nn.Flatten(),
                nn.Sigmoid()
            ))
    
    #forward pass, calls the class init
    def forward(self, x):
        for layer in self.layers:
            output = layer(x)
        return output

In [36]:
dis = Discriminator(4,64)
dis

Discriminator(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (2): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (3): Sequential(
      (0): Conv2d(192, 1, kernel_size=(3, 3), stride=(1, 1))
      (1): Flatten(start_dim=1, end_dim=-1)
      (2): Sigmoid()
    )
  )
)

In [8]:
some_size = 256 #same output from discriminator: 256x1x1

In [83]:
class Generator(nn.Module):
    def __init__(self, nl, hs):
        super().__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Sequential(
            nn.ConvTranspose2d((hs*(2**nl)), (int)((hs*(2**nl))/2), 3, 1, 0), #last hidden size, the one before
            nn.BatchNorm2d((hs*(nl-1))/2), 
            nn.ReLU()
        ))
        
        for i in range(nl-1, 0, -1): #decrementing through the number of layers, this will help with the size in each layer.
            self.layer.append(nn.Sequential(
                nn.ConvTranspose2d((int)((hs*(2**nl))/(2**i)), (int)((hs*(2**nl))/(2*2**i)),3, 1, 0),
                nn.BatchNorm2d((hs*(nl-1))/2), 
                nn.ReLU()
            ))
        self.layer.append(nn.Sequential(
                nn.ConvTranspose2d(hs, 3, 3, 1, 0),
                nn.Sigmoid()
            ))
        
    def forward(self, input):
        output = self.layers(input)
        return output

In [84]:
gen = Generator(4, 64)

TypeError: empty() received an invalid combination of arguments - got (float, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of SymInts size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


Notes to self: Re-check prof. alvarez notes for convtranspose2d.

Great additional article for convtranspose2d: https://medium.com/apache-mxnet/transposed-convolutions-explained-with-ms-excel-52d13030c7e8. Additionally explains why convtranspose2d is not the same as deconvolution.


<br> The generator and discriminator should have the same "in-reverse" architecture to one another:

In [21]:
#creating instances of our two models;
dis = Discriminator()
gen = Generator()
loss = torch.nn.BCELoss() #binary cross entropy: either its classified as real or fake.

In [28]:
#some hyperparam init: #wandb script here: https://wandb.ai/quickstart, https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Organizing_Hyperparameter_Sweeps_in_PyTorch_with_W%26B.ipynb#scrollTo=UsM_GTz-742w
bs = 5000
lr = 0.002
#dis_optim = torch.optim.AdamW(dis.parameters(), lr= lr, betas=(0.9,0.99))
optimizer = torch.optim.AdamW

In [78]:
#finding layers for generator:
hs=64
nl=4
print((hs*(2**nl)), (int)((hs*(2**nl))/2))
for i in range(1, nl-1):
    print((int)((hs*(2**nl))/(2**i)), (int)((hs*(2**nl))/(2*2**i)))

print(hs, 3)

1024 512
512 256
256 128
64 3
